In [1]:
import pandas as pd

In [70]:
tags = pd.read_xml("Tags.cri", xpath=".//Item")
tags["Name"] = tags["Source"].str.split(".").str[1]

In [71]:
tags

,Target,TargetProperty,Source,Name
0,Tags,VL3_Man,Controller1.DataItem1,DataItem1
1,Tags,VL3_OpnCmdMan,Controller1.DataItem2,DataItem2
2,Tags,VL3_OpnCmdAut,Controller1.DataItem3,DataItem3
3,Tags,VL3_OpnCmd,Controller1.DataItem4,DataItem4
4,Tags,VL4_Man,Controller1.DataItem5,DataItem5
...,...,...,...,...
185,Tags,State,Controller1.DataItem188,DataItem188
186,Tags,ButtonHMI_Auto,Controller1.DataItem34,DataItem34
187,Tags,BubblerTempSet,Controller1.DataItem67,DataItem67
188,Tags,StackTempSet,Controller1.DataItem114,DataItem114


In [67]:
import xml.etree.ElementTree as ET

root = ET.parse("Controller1.neo").getroot()
dataitems = root.find(".//DataItems")

res = []

for node in dataitems:
    obj = node[0]
    Name = obj.find(".//Parameter[@Name='Name']")
    ItemId = obj.find(".//Parameter[@Name='ItemID']")
    DataType = obj.find(".//Parameter[@Name='DataType']")
    res.append({"Name": Name[0].get("primitive.value"), "Addr": ItemId[0].get("primitive.value"), "Type": DataType[0].get("primitive.value")})

cont = pd.DataFrame(res)

In [68]:
cont

,Name,Addr,Type
0,DataItem116,DB260.DBX8.1,BOOL
1,DataItem1,DB180.DBX0.0,BOOL
2,DataItem2,DB180.DBX0.1,BOOL
3,DataItem3,DB180.DBX0.2,BOOL
4,DataItem4,DB180.DBX2.0,BOOL
...,...,...,...
185,DataItem34,DB3.DBX0.5,BOOL
186,DataItem45,DB49.DBX8.4,BIT
187,DataItem67,DB24.DBD10,FLOAT
188,DataItem114,DB24.DBD14,FLOAT


In [83]:
res = pd.merge(tags, cont, on="Name")
res = res.rename(columns = {"TargetProperty": "name", "Type": "type", "Addr": "id"})
res = res.drop(["Target", "Source", "Name"], axis = 1)
res

,name,id,type
0,VL3_Man,DB180.DBX0.0,BOOL
1,VL3_OpnCmdMan,DB180.DBX0.1,BOOL
2,VL3_OpnCmdAut,DB180.DBX0.2,BOOL
3,VL3_OpnCmd,DB180.DBX2.0,BOOL
4,VL4_Man,DB181.DBX0.0,BOOL
...,...,...,...
185,State,DB24.DBW2,INT16
186,ButtonHMI_Auto,DB3.DBX0.5,BOOL
187,BubblerTempSet,DB24.DBD10,FLOAT
188,StackTempSet,DB24.DBD14,FLOAT


In [85]:
res.to_json("S7_registers.json", orient="records")